In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import numpy as np
from datetime import datetime
import yfinance as yf
import os
import glob
import shutil

In [ ]:
numblist = [1,2,3,4,5,6,7]

sectorlist = ["automobiles", "banks", "capital-goods", "commercial-services", 
"consumer-durables", "consumer-retailing", "consumer-services", "diversified-financials",
"energy", "food-beverage-tobacco", "healthcare", "household", "insurance", "materials", "media", 
"pharmaceuticals-biotech", "real-estate", "retail", "semiconductors", "software",
"tech", "telecom", "transportation", "utilities"]

urllist = []

for sector in sectorlist:
    for number in numblist:
        urllist.append('https://simplywall.st/stocks/us/' + sector + '/market-cap-large?page=' + str(number))


count = 0
count2 = 0
df = pd.DataFrame(columns=['href','ticker'])

for i in range(0, len(urllist)+1):
    if count == 7:   
            count = 0
            df.to_csv('hrefs/ref_' + (sectorlist[count2]) + '.csv', index=False)
            count2 += 1
            df = pd.DataFrame(columns=['href','ticker'])
        
    try:
        url = urllist[i]
        print(url)
        count+=1
        html_text = requests.get(url).text
        
        soup = BeautifulSoup(html_text, "html.parser")
        body = soup.find('tbody').find_all('tr')

        for j in body:
            temp = j.find_all('td')[1]
            insert_row = {"href": (temp.a['href']), "ticker": ''.join(temp.find('b').contents)}
            df = pd.concat([df, pd.DataFrame([insert_row])])

    except:
        continue 


In [ ]:
# sectorlist = ["automobiles", "banks", "capital-goods", "commercial-services", 
# "consumer-durables", "consumer-retailing", "consumer-services", "diversified-financials",
# "energy", "food-beverage-tobacco", "healthcare", "household", "insurance", "materials", "media", 
# "pharmaceuticals-biotech", "real-estate", "retail", "semiconductors", "software",
# "tech", "telecom", "transportation", "utilities"]

sectorlist = ["pharmaceuticals-biotech", "real-estate", "retail", "semiconductors", "software",
"tech", "telecom", "transportation", "utilities"]

for sector in sectorlist:
    df = pd.read_csv('hrefs/ref_' + (sector) + '.csv')
    
    hrefs = df["href"]
    df_fair_value = pd.DataFrame()
    for href in hrefs:
        time.sleep(1.5)
        url = "https://simplywall.st" + str(href) + "/valuation"
        print(url)
        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, "html.parser")
        body = soup.find_all('blockquote')
        count = 0
        fair_value = np.NaN
        for i in body:
            count += 1
            if count < 25:
                try:
                    q = ''.join(i.find('p').find_all('span')[3].contents)
                    if q[2] == "$":
                        fair_value = q
                except:
                    continue
            else:
                break
        insert_row = {"fair_value": fair_value}
        
        df_fair_value = pd.concat([df_fair_value, pd.DataFrame([insert_row])])
        print(fair_value)
    df_initial = pd.read_csv('hrefs/ref_'+sector+'.csv')
    
    
    #df_initial = df_initial.drop('fair_value', axis=1)
   
    temp2 = df_fair_value.to_numpy()
    
    df_initial.insert(loc=2, column="fair_value", value=temp2)
    df_initial.to_csv('hrefs/ref_' + (sector) + '.csv', index=False)



In [ ]:
pd.set_option('display.max_rows', df_next.shape[0]+1)

df_next


In [ ]:
df_initial = pd.read_csv('hrefs/ref_'+"automobiles"+'.csv')
temp2 = df_fair_value.to_numpy()
df_initial.insert(loc=2, column="fair_value", value=temp2)
df_initial.to_csv('hrefs/ref_' + (sector) + '_copy.csv', index=False)

df_initial

In [ ]:
#Deletes all duplicates in dataframe
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

sectorlist = ["automobiles", "banks", "capital-goods", "commercial-services", 
"consumer-durables", "consumer-retailing", "consumer-services", "diversified-financials",
"energy", "food-beverage-tobacco", "healthcare", "household", "insurance", "materials", "media", 
"pharmaceuticals-biotech", "real-estate", "retail", "semiconductors", "software",
"tech", "telecom", "transportation", "utilities"]

for sector in sectorlist:
    df_initial = pd.read_csv('hrefs/ref_'+sector+'.csv')
    print(df_initial)
    df_initial = df_initial.drop_duplicates()
    df_initial.to_csv('hrefs/ref_'+sector+'.csv', index=False)
    

In [5]:
def find_transcript(ticker, date_yr, date_qtr, i):
    try:
        url = 'https://roic.ai/transcripts/' + ticker + '?y=' + str(date_yr) + '&q=' + str(date_qtr)
        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, "html.parser")
        # site below is used to visualise JSON files (indented)
        # https://jsonformatter.org/
        # finds the 15th <script/> and strips the unnecsssary data so it can be read in JSON
        script = soup.find_all('script')[15].text.strip()  
        data = json.loads(script)
        transcript_data = data['props']['pageProps']['transcriptdata']['content']  # loads the transcript content
        exact_date = datetime.strptime(data['props']['pageProps']['data']['data']['earningscalls'][i]['date'], '%Y-%m-%d %H:%M:%S')
        #exact_date = 'day 0', earnings transcript release date
        ntr = yf.Ticker(ticker)
    except:
        transcript_data = np.NaN
        exact_date = np.NaN
        
    return transcript_data, exact_date

def ticker_transcript(ticker):
    #loops around each ticker, feature extracts each by calling relevant functions below
    try:
        url = 'https://roic.ai/transcripts/' + ticker

        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, "html.parser")

        script = soup.find_all('script')[15].text.strip()
        # finds json data (i.e. transcript)
        data = json.loads(script)
        first_er_date = data['props']['pageProps']['data']['data']['earningscalls'][0]
        last_er_date = data['props']['pageProps']['data']['data']['earningscalls'][-1]
        # reads the json data and finds the first and last date for which the transcript is held and its set as limits
        # perhaps I worded it badly...
        # first_er_date_yr represents the most recent transcript's date (i.e. 2022)
        # last_er_date represents the last transcript's date (e.g. 2006)

        first_er_date_yr = first_er_date['year'] 
        first_er_date_qtr = first_er_date['quarter']
        last_er_date_yr = last_er_date['year']
        # calculates the year differences between the dates of first and last transcript so it can be looped
        yr_difference = first_er_date_yr-last_er_date_yr
        date_yr = first_er_date_yr
        date_qtr = first_er_date_qtr

        arr_ticker_infos = np.empty((0, 4), str)
        arr_full_transcript = np.array([], str) #array of all of the transcripts, every year and quarter
        q = -1

        for i in range(0, yr_difference + 1):
            #there are two seperate loops below:
            if date_yr == first_er_date_yr:
                # the first loop is to loop through the most recent transcript as the quarter does not end with 4
                # as of when this programme is written, the most recent transcript is Q2 of 2022
                for j in range(0, first_er_date_qtr):
                    q += 1
                    #finds the relevant 13 initial dataset as described in page 2 of the Doc
                    #it finds them by calling the function find_transcript() located below
                    full_transcript, exact_date = find_transcript(ticker, date_yr, date_qtr, q)
                    arr_full_transcript = np.append(arr_full_transcript, full_transcript)
                    arr_ticker_infos = np.append(arr_ticker_infos, np.array([[ticker, date_yr, date_qtr, exact_date]]), axis=0)
                    date_qtr -= 1
                    if date_qtr == 0:
                        date_qtr = 4
            else:
                # the second loop loops through the rest of the transcripts
                for j in range(0, 4):
                    q += 1
                    
                    full_transcript, exact_date = find_transcript(ticker, date_yr, date_qtr, q)
                    arr_full_transcript = np.append(arr_full_transcript, full_transcript)
                    arr_ticker_infos = np.append(arr_ticker_infos, np.array([[ticker, date_yr, date_qtr, exact_date]]), axis=0)

                    date_qtr -= 1
                    if date_qtr == 0:
                        date_qtr = 4
            date_yr -= 1
    except:
        arr_full_transcript = np.NaN
        arr_ticker_infos = np.NaN
        
    return arr_full_transcript, arr_ticker_infos

In [ ]:
#Finds transcript and creates the .csv files
for sector in sectorlist:
    df_stock_list = pd.read_csv('hrefs/ref_'+sector+'.csv')['ticker']
    for stock in df_stock_list:
        try:
            try: 
                filelist = os.listdir("sectors/"+sector)
            except:
                filelist = []

            if stock not in filelist:
                print(stock)

                arr_full_transcript, arr_ticker_infos = ticker_transcript(stock)
                FOLDERPATH = 'sectors/' + sector + '/' + str(stock)
                if not os.path.exists(FOLDERPATH):
                    os.makedirs(FOLDERPATH)
                for i in range(0,len(arr_ticker_infos)):
                    year = arr_ticker_infos[i][1]
                    quarter = arr_ticker_infos[i][2]
                    release_date = arr_ticker_infos[i][3]
                    single_transcript = arr_full_transcript[i]
                    df_transcript_details = pd.DataFrame([year, quarter, release_date, single_transcript])
                    FILEPATH = 'sectors/' + sector + '/' + str(stock) + '/' + str(stock) + str(year) + str(quarter) + ".csv"
                    df_transcript_details.to_csv(FILEPATH, header=False, index=False)
        except:
            continue

In [137]:

sum=0
for sector in sectorlist:
    filelist = os.listdir("sectors/"+sector)
    try:
        filelist.remove('.DS_Store')
    except:
        pass
    sum += len(filelist)
    
print(sum) #amount of stocks

2463


In [ ]:
#deletes all csv files with NaN values as transcript data
for sector in sectorlist:
    filelist = os.listdir("sectors/"+sector)
    try:
        filelist.remove('.DS_Store')
    except:
        pass
    for stock in filelist:
        sector_files = glob.glob('sectors/'+str(sector)+'/'+str(stock)+'/'+str(stock)+'20*[0-9]**[0-9]*[1-4].*')
        for directory_link in sector_files:
            try:
                df_transcript = pd.read_csv(directory_link)
                my_transcript = df_transcript.loc[2]
                if str(my_transcript[0]) == "nan":
                    print(directory_link)
                    os.remove(directory_link)

            except:
                pass
            


In [136]:
sectorlist = ["automobiles", "banks", "capital-goods", "commercial-services", 
"consumer-durables", "consumer-retailing", "consumer-services", "diversified-financials",
"energy", "food-beverage-tobacco", "healthcare", "household", "insurance", "materials", "media", 
"pharmaceuticals-biotech", "real-estate", "retail", "semiconductors", "software",
"tech", "telecom", "transportation", "utilities"]

In [ ]:
#deletes stock directories with less than 5 earnings calls
for sector in sectorlist:
    filelist = os.listdir("sectors/"+sector)
    try:
        filelist.remove('.DS_Store')
    except:
        pass
    for stock in filelist:
        sector_files = glob.glob('sectors/'+sector+'/'+str(stock))
        for directory_link in sector_files:
            if len(os.listdir(directory_link)) < 5:
                print(directory_link)
                shutil.rmtree(directory_link, ignore_errors=True)




In [13]:
# sorts directory into a list
sectorlist = ["automobiles", "banks", "capital-goods", "commercial-services", 
"consumer-durables", "consumer-retailing", "consumer-services", "diversified-financials",
"energy", "food-beverage-tobacco", "healthcare", "household", "insurance", "materials", "media", 
"pharmaceuticals-biotech", "real-estate", "retail", "semiconductors", "software",
"tech", "telecom", "transportation", "utilities"]

for sector in sectorlist:
    
    filelist = os.listdir("sectors/"+sector)
    try:
        filelist.remove('.DS_Store')
    except:
        pass
    for stock in filelist:
        all_files = glob.glob('sectors/'+str(sector)+'/'+str(stock)+'/'+str(stock)+'20*[0-9]**[0-9]*[1-4].*')
        all_files.sort(reverse=True)
        print(all_files)

        break
    break


['sectors/automobiles/XPEL/XPEL20151.csv', 'sectors/automobiles/XPEL/XPEL20152.csv', 'sectors/automobiles/XPEL/XPEL20153.csv', 'sectors/automobiles/XPEL/XPEL20154.csv', 'sectors/automobiles/XPEL/XPEL20161.csv', 'sectors/automobiles/XPEL/XPEL20162.csv', 'sectors/automobiles/XPEL/XPEL20163.csv', 'sectors/automobiles/XPEL/XPEL20164.csv', 'sectors/automobiles/XPEL/XPEL20171.csv', 'sectors/automobiles/XPEL/XPEL20172.csv', 'sectors/automobiles/XPEL/XPEL20173.csv', 'sectors/automobiles/XPEL/XPEL20174.csv', 'sectors/automobiles/XPEL/XPEL20181.csv', 'sectors/automobiles/XPEL/XPEL20182.csv', 'sectors/automobiles/XPEL/XPEL20183.csv', 'sectors/automobiles/XPEL/XPEL20184.csv', 'sectors/automobiles/XPEL/XPEL20191.csv', 'sectors/automobiles/XPEL/XPEL20192.csv', 'sectors/automobiles/XPEL/XPEL20193.csv', 'sectors/automobiles/XPEL/XPEL20194.csv', 'sectors/automobiles/XPEL/XPEL20201.csv', 'sectors/automobiles/XPEL/XPEL20202.csv', 'sectors/automobiles/XPEL/XPEL20204.csv', 'sectors/automobiles/XPEL/XPEL202

In [12]:
url = pd.read_csv("sectors/automobiles/XPEL/XPEL20151.csv")
print(url)


                                                2015
0                                                  1
1                                2015-03-31 17:00:00
2  Operator: Greetings, and welcome to the XPEL T...


In [ ]:
compare earnings transcript sentiments between companies in the same sector and see variances, then rank them

https://www.benzinga.com/quote/AAPL


how much it missed/beats earnings

compare analyst questions with responce

have a list of all operator/speaker names to delete
